In [1]:
import pandas as pd
df = pd.read_csv('spear_phishing_train.csv')
df.head()

,Unnamed: 0,Tweet,following,followers,actions,is_retweet,location,Type,Unnamed: 7,Type_num
0,1,'@realDonaldTrump @USNavy RIP TO HEROES',42096.0,61060.0,5001.0,0.0,"South Padre Island, Texas",Spam,NaN,1
1,4,#DidYouKnow ► Mahatma Gandhi made a brief visi...,17800.0,35100.0,NaN,0.0,"Nottingham, England",Quality,NaN,0
2,6,please don't talk about me like that i'm only ...,0.0,6200000.0,NaN,1.0,"los angeles, ca",Quality,NaN,0
3,22,Need I remind everyone that the first lines yo...,0.0,275000.0,80.0,0.0,"Johannesburg, South Africa",Quality,NaN,0
4,23,Watch: #MonstaX Goes “#AllIn” For Their Comeba...,0.0,2250000.0,NaN,0.0,Lord Juja ❤️,Quality,NaN,0


In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.Tweet, df.Type_num, test_size = 0.30)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train_count = vectorizer.fit_transform(X_train.values)
X_train_count.toarray()[:4]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [4]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

clf1 = MultinomialNB()
clf2 = DecisionTreeClassifier(random_state=0, max_depth=50)
clf3 = KNeighborsClassifier(n_neighbors=1)
clf4 = LogisticRegression(random_state=0)
clf5 = svm.SVC()
clf6 = RandomForestClassifier(n_estimators = 140)

clf1.fit(X_train_count, y_train)
clf2.fit(X_train_count, y_train)
clf3.fit(X_train_count, y_train)
clf4.fit(X_train_count, y_train)
clf5.fit(X_train_count, y_train)
clf6.fit(X_train_count, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=140,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [5]:
from sklearn.svm.libsvm import predict_proba
from sklearn.multioutput import MultiOutputClassifier

X_test_count = vectorizer.transform(X_test)
pred1 = clf1.predict_proba(X_test_count)
pred2 = clf2.predict_proba(X_test_count)
pred3 = clf3.predict_proba(X_test_count)
pred4 = clf4.predict_proba(X_test_count)
pred6 = clf6.predict_proba(X_test_count)

final_pred = (pred1 + pred2 + pred3 + pred4 + pred6)/5
final_pred

C:\Users\samra\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.svm.libsvm module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)


array([[9.85207333e-01, 1.47926665e-02],
       [9.99694783e-01, 3.05217181e-04],
       [9.95282458e-01, 4.71754194e-03],
       ...,
       [7.80690079e-01, 2.19309921e-01],
       [9.87223830e-01, 1.27761696e-02],
       [7.47405300e-01, 2.52594700e-01]])

In [6]:
from sklearn.metrics import accuracy_score

y_pred = []
for x in final_pred :
    if x[0] > 0.5 :
        y_pred.append(0)
    else :
        y_pred.append(1)

accuracy_score(y_pred, y_test)

0.9496644295302014